In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base 
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract
import sqlite3

In [2]:
books_df = pd.read_csv("Resources/Nilim-Kaggle/books.csv", error_bad_lines=False)
books_df.dropna(inplace = True) 
books_df.head()

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [3]:
books_df["publisher"].value_counts()

Vintage                                                   318
Penguin Books                                             261
Penguin Classics                                          184
Mariner Books                                             150
Ballantine Books                                          144
                                                         ... 
Klett                                                       1
Dell Yearling                                               1
Insel                                                       1
Random House UK Ltd (A Division of Random House Group)      1
Piper Taschenbuch                                           1
Name: publisher, Length: 2290, dtype: int64

In [4]:
books_df["language_code"].value_counts()

eng      8908
en-US    1408
spa       218
en-GB     214
fre       144
ger        99
jpn        46
mul        19
zho        14
grc        11
por        10
en-CA       7
ita         5
enm         3
lat         3
swe         2
rus         2
ara         1
wel         1
nl          1
glg         1
ale         1
gla         1
msa         1
srp         1
nor         1
tur         1
Name: language_code, dtype: int64

In [5]:
#en-CA = English (Canada), en-GB =English (United Kingdom), en-US= English (United States) update all to eng. 
eng_books_df= books_df.loc[(books_df["language_code"] == "en-CA") | (books_df["language_code"] == "en-GB")| (books_df["language_code"] == "en-US")].replace(['en-CA','en-GB','en-US'],'eng')
eng_books_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,0976540606,9780976540601,eng,152,19,1,4/26/2005,Nimble Books
24,35,The Lord of the Rings (The Lord of the Rings ...,J.R.R. Tolkien/Alan Lee,4.50,0618260587,9780618260584,eng,1216,1618,140,10/1/2002,Houghton Mifflin Harcourt
32,55,Hatchet Jobs: Writings on Contemporary Fiction,Dale Peck,3.45,1595580271,9781595580276,eng,228,99,16,11/1/2005,The New Press
40,68,The Known World,Edward P. Jones/Kevin R. Free,3.83,006076273X,9780060762735,eng,14,55,12,6/15/2004,HarperAudio
45,75,Uncommon Carriers,John McPhee,3.95,0374280398,9780374280390,eng,248,1630,203,5/16/2006,Farrar Straus Giroux


In [6]:
books_df["language_code"].value_counts()

eng      8908
en-US    1408
spa       218
en-GB     214
fre       144
ger        99
jpn        46
mul        19
zho        14
grc        11
por        10
en-CA       7
ita         5
enm         3
lat         3
swe         2
rus         2
ara         1
wel         1
nl          1
glg         1
ale         1
gla         1
msa         1
srp         1
nor         1
tur         1
Name: language_code, dtype: int64

In [7]:
books_df.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [8]:
books_df['publication_date'] = pd.to_datetime(books_df['publication_date'], dayfirst=True, errors='coerce')

In [9]:
books_df.dtypes

bookID                         int64
title                         object
authors                       object
average_rating               float64
isbn                          object
isbn13                         int64
language_code                 object
  num_pages                    int64
ratings_count                  int64
text_reviews_count             int64
publication_date      datetime64[ns]
publisher                     object
dtype: object

##### Database Setup 

In [10]:
conn = sqlite3.connect('books_test_db')
c = conn.cursor()

In [11]:
c.execute('''CREATE TABLE books(bookID, title, authors, average_rating, isbn, isbn13, language_code, num_pages, ratings_count, text_reviews_count, publication_date, publisher)''')
conn.commit()

OperationalError: table books already exists

In [ ]:
books_df.to_sql('books', conn, if_exists='replace', index = False)

In [ ]:
engine = create_engine("sqlite:///books_test_db.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# 4. Create a DataFrame from . 
june_df = pd.DataFrame(june_results, columns=['date','June Temps'])
print(june_df)

In [ ]:
# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
ratings_results = session.query().filter(extract('month', Measurement.date) == 6).all()